In [1]:
!pip install ortools

     |████████████████████████████████| 16.3 MB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 12.5 MB/s eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.16.0
    Uninstalling protobuf-3.16.0:
      Successfully uninstalled protobuf-3.16.0


In [3]:
!pip install pyproj

     |████████████████████████████████| 7.8 MB 4.0 MB/s eta 0:00:01


In [1]:
import pyproj
from ortools.constraint_solver import pywrapcp
import pandas as pd
import openrouteservice
from openrouteservice import convert
import time

In [2]:
shizuoka_imgs = pd.read_csv('./shizuoka_imgs.csv',header=None, names=['id', 'lat', 'lon'])
shizuoka_imgs

,id,lat,lon
0,4.077459e+09,35.014708,138.922408
1,8.727410e+09,34.973188,138.383231
2,7.571857e+09,34.996816,138.457260
3,1.683536e+10,34.968594,138.389080
4,2.864332e+09,34.982612,138.417606
...,...,...,...
89,5.588371e+09,34.962291,138.086119
90,7.911333e+09,34.972567,138.388130
91,3.778222e+10,34.907863,139.096934
92,2.614825e+10,34.907863,139.096934


In [3]:
key = ["5b3ce3597851110001cf62489d3c30f6edcf468dbf5f4e18fabff66e","5b3ce3597851110001cf624833efcdb5681f42acbee22fe105edd2f5","5b3ce3597851110001cf6248f505d6707c1b406096f53089eb956a5b"]
client = openrouteservice.Client(key=key[0])
key

['5b3ce3597851110001cf62489d3c30f6edcf468dbf5f4e18fabff66e',
 '5b3ce3597851110001cf624833efcdb5681f42acbee22fe105edd2f5',
 '5b3ce3597851110001cf6248f505d6707c1b406096f53089eb956a5b']

In [4]:
distance = [[0 for i in range(len(shizuoka_imgs))] for j in range(len(shizuoka_imgs))]
duration = [[0 for i in range(len(shizuoka_imgs))] for j in range(len(shizuoka_imgs))]

In [5]:
index_i = 0
index_key = 0

In [8]:
for i in range(len(shizuoka_imgs)):
    for j in range(len(shizuoka_imgs)):
        if (i > j) and (i >= 17):    
            p1 = shizuoka_imgs.iloc[i,1], shizuoka_imgs.iloc[i,2]
            p2 = shizuoka_imgs.iloc[j,1], shizuoka_imgs.iloc[j,2]
            p1r = tuple(reversed(p1))
            p2r = tuple(reversed(p2))
            if (p1r != p2r):
                mean_lat = (p1[0] + p2[0]) / 2
                mean_long = (p1[1] + p2[1]) / 2

                # 経路計算 (Directions V2)
                print(i, j,p1r,p2r)
                routedict = client.directions((p1r, p2r))
                try:
                    distance[i][j] = routedict["routes"][0]["summary"]["distance"]
                    duration[i][j] = routedict["routes"][0]["summary"]["duration"]
                except KeyError as instance:
                    print(instance)
                time.sleep(1.5)
                index_i += 1
                if index_i // 1900 != index_key:
                    index_key = index_i // 1900
                    client = openrouteservice.Client(key=key[index_key])

17 0 (138.462667, 34.980924) (138.922408, 35.014708)
17 1 (138.462667, 34.980924) (138.383231, 34.973188)
17 2 (138.462667, 34.980924) (138.45726, 34.996816)
17 3 (138.462667, 34.980924) (138.38908, 34.968594)
17 4 (138.462667, 34.980924) (138.417606, 34.982612)
17 5 (138.462667, 34.980924) (137.970685, 34.743446)
17 7 (138.462667, 34.980924) (138.445501, 34.953985)
17 8 (138.462667, 34.980924) (138.606605, 35.120891)
17 13 (138.462667, 34.980924) (138.47949, 35.022967)
17 14 (138.462667, 34.980924) (138.389633, 34.971105)
17 15 (138.462667, 34.980924) (138.389797, 34.972241)
17 16 (138.462667, 34.980924) (138.389811, 34.972277)
18 0 (138.389224, 34.971519) (138.922408, 35.014708)
18 1 (138.389224, 34.971519) (138.383231, 34.973188)
18 2 (138.389224, 34.971519) (138.45726, 34.996816)
18 3 (138.389224, 34.971519) (138.38908, 34.968594)
18 4 (138.389224, 34.971519) (138.417606, 34.982612)
18 5 (138.389224, 34.971519) (137.970685, 34.743446)
18 6 (138.389224, 34.971519) (138.462667, 34.98

24 20 (138.963501, 35.310001) (138.408508, 34.97586)
24 21 (138.963501, 35.310001) (138.388205, 34.9735)
24 22 (138.963501, 35.310001) (138.389113, 34.971444)
24 23 (138.963501, 35.310001) (138.387094, 34.971925)
25 0 (138.375825, 34.983638) (138.922408, 35.014708)
25 1 (138.375825, 34.983638) (138.383231, 34.973188)
25 2 (138.375825, 34.983638) (138.45726, 34.996816)
25 3 (138.375825, 34.983638) (138.38908, 34.968594)
25 4 (138.375825, 34.983638) (138.417606, 34.982612)
25 5 (138.375825, 34.983638) (137.970685, 34.743446)
25 6 (138.375825, 34.983638) (138.462667, 34.980924)
25 7 (138.375825, 34.983638) (138.445501, 34.953985)
25 8 (138.375825, 34.983638) (138.606605, 35.120891)
25 9 (138.375825, 34.983638) (138.462667, 34.980924)
25 10 (138.375825, 34.983638) (138.462667, 34.980924)
25 11 (138.375825, 34.983638) (138.462667, 34.980924)
25 12 (138.375825, 34.983638) (138.462667, 34.980924)
25 13 (138.375825, 34.983638) (138.47949, 35.022967)
25 14 (138.375825, 34.983638) (138.389633, 3

30 16 (138.965164, 35.311001) (138.389811, 34.972277)
30 17 (138.965164, 35.311001) (138.462667, 34.980924)
30 18 (138.965164, 35.311001) (138.389224, 34.971519)
30 19 (138.965164, 35.311001) (138.388533, 34.972477)
30 20 (138.965164, 35.311001) (138.408508, 34.97586)
30 21 (138.965164, 35.311001) (138.388205, 34.9735)
30 22 (138.965164, 35.311001) (138.389113, 34.971444)
30 23 (138.965164, 35.311001) (138.387094, 34.971925)
30 24 (138.965164, 35.311001) (138.963501, 35.310001)
30 25 (138.965164, 35.311001) (138.375825, 34.983638)
30 26 (138.965164, 35.311001) (137.602829, 34.688835)
30 27 (138.965164, 35.311001) (138.84221, 35.100529)
30 28 (138.965164, 35.311001) (137.602997, 34.688835)
30 29 (138.965164, 35.311001) (137.602661, 34.688835)
31 0 (137.656326, 34.698334) (138.922408, 35.014708)
31 1 (137.656326, 34.698334) (138.383231, 34.973188)
31 2 (137.656326, 34.698334) (138.45726, 34.996816)
31 3 (137.656326, 34.698334) (138.38908, 34.968594)
31 4 (137.656326, 34.698334) (138.4176

35 11 (138.962173, 35.313835) (138.462667, 34.980924)
35 12 (138.962173, 35.313835) (138.462667, 34.980924)
35 13 (138.962173, 35.313835) (138.47949, 35.022967)
35 14 (138.962173, 35.313835) (138.389633, 34.971105)
35 15 (138.962173, 35.313835) (138.389797, 34.972241)
35 16 (138.962173, 35.313835) (138.389811, 34.972277)
35 17 (138.962173, 35.313835) (138.462667, 34.980924)
35 18 (138.962173, 35.313835) (138.389224, 34.971519)
35 19 (138.962173, 35.313835) (138.388533, 34.972477)
35 20 (138.962173, 35.313835) (138.408508, 34.97586)
35 21 (138.962173, 35.313835) (138.388205, 34.9735)
35 22 (138.962173, 35.313835) (138.389113, 34.971444)
35 23 (138.962173, 35.313835) (138.387094, 34.971925)
35 24 (138.962173, 35.313835) (138.963501, 35.310001)
35 25 (138.962173, 35.313835) (138.375825, 34.983638)
35 26 (138.962173, 35.313835) (137.602829, 34.688835)
35 27 (138.962173, 35.313835) (138.84221, 35.100529)
35 28 (138.962173, 35.313835) (137.602997, 34.688835)
35 29 (138.962173, 35.313835) (13

39 20 (138.926668, 35.370959) (138.408508, 34.97586)
39 21 (138.926668, 35.370959) (138.388205, 34.9735)
39 22 (138.926668, 35.370959) (138.389113, 34.971444)
39 23 (138.926668, 35.370959) (138.387094, 34.971925)
39 24 (138.926668, 35.370959) (138.963501, 35.310001)
39 25 (138.926668, 35.370959) (138.375825, 34.983638)
39 26 (138.926668, 35.370959) (137.602829, 34.688835)
39 27 (138.926668, 35.370959) (138.84221, 35.100529)
39 28 (138.926668, 35.370959) (137.602997, 34.688835)
39 29 (138.926668, 35.370959) (137.602661, 34.688835)
39 30 (138.926668, 35.370959) (138.965164, 35.311001)
39 31 (138.926668, 35.370959) (137.656326, 34.698334)
39 32 (138.926668, 35.370959) (137.645828, 34.698002)
39 33 (138.926668, 35.370959) (138.965164, 35.311001)
39 34 (138.926668, 35.370959) (138.390325, 34.973055)
39 35 (138.926668, 35.370959) (138.962173, 35.313835)
39 36 (138.926668, 35.370959) (138.963501, 35.310001)
39 37 (138.926668, 35.370959) (138.926496, 35.371694)
39 38 (138.926668, 35.370959) (1

...

86 11 (138.087661, 34.953336) (138.462667, 34.980924)
86 12 (138.087661, 34.953336) (138.462667, 34.980924)
86 13 (138.087661, 34.953336) (138.47949, 35.022967)
86 14 (138.087661, 34.953336) (138.389633, 34.971105)
86 15 (138.087661, 34.953336) (138.389797, 34.972241)
86 16 (138.087661, 34.953336) (138.389811, 34.972277)
86 17 (138.087661, 34.953336) (138.462667, 34.980924)
86 18 (138.087661, 34.953336) (138.389224, 34.971519)
86 19 (138.087661, 34.953336) (138.388533, 34.972477)
86 20 (138.087661, 34.953336) (138.408508, 34.97586)
86 21 (138.087661, 34.953336) (138.388205, 34.9735)
86 22 (138.087661, 34.953336) (138.389113, 34.971444)
86 23 (138.087661, 34.953336) (138.387094, 34.971925)
86 24 (138.087661, 34.953336) (138.963501, 35.310001)
86 25 (138.087661, 34.953336) (138.375825, 34.983638)
86 26 (138.087661, 34.953336) (137.602829, 34.688835)
86 27 (138.087661, 34.953336) (138.84221, 35.100529)
86 28 (138.087661, 34.953336) (137.602997, 34.688835)
86 29 (138.087661, 34.953336) (13

87 75 (138.442702, 35.001422) (138.38808, 34.98008)
87 76 (138.442702, 35.001422) (138.149557, 35.167739)
87 77 (138.442702, 35.001422) (138.836116, 34.971821)
87 78 (138.442702, 35.001422) (138.095444, 35.051286)
87 79 (138.442702, 35.001422) (138.93671, 35.092945)
87 80 (138.442702, 35.001422) (138.858947, 35.098282)
87 81 (138.442702, 35.001422) (138.858947, 35.098282)
87 82 (138.442702, 35.001422) (138.858947, 35.098282)
87 83 (138.442702, 35.001422) (139.096923, 34.907872)
87 84 (138.442702, 35.001422) (138.085725, 34.95008)
87 85 (138.442702, 35.001422) (138.390569, 34.972062)
87 86 (138.442702, 35.001422) (138.087661, 34.953336)
88 0 (137.612427, 34.79008) (138.922408, 35.014708)
88 1 (137.612427, 34.79008) (138.383231, 34.973188)
88 2 (137.612427, 34.79008) (138.45726, 34.996816)
88 3 (137.612427, 34.79008) (138.38908, 34.968594)
88 4 (137.612427, 34.79008) (138.417606, 34.982612)
88 5 (137.612427, 34.79008) (137.970685, 34.743446)
88 6 (137.612427, 34.79008) (138.462667, 34.98

89 53 (138.086119, 34.962291) (138.390569, 34.972062)
89 54 (138.086119, 34.962291) (138.390569, 34.972062)
89 55 (138.086119, 34.962291) (138.52391205768, 34.995366104263)
89 56 (138.086119, 34.962291) (138.52391205768, 34.995366104263)
89 57 (138.086119, 34.962291) (138.385848, 34.973891)
89 58 (138.086119, 34.962291) (138.926496, 35.371694)
89 59 (138.086119, 34.962291) (138.390569, 34.972062)
89 60 (138.086119, 34.962291) (138.552961, 35.097861)
89 61 (138.086119, 34.962291) (138.311462, 35.120281)
89 62 (138.086119, 34.962291) (138.658661, 35.141402)
89 63 (138.086119, 34.962291) (137.603745, 34.693005)
89 64 (138.086119, 34.962291) (138.673691, 35.143811)
89 65 (138.086119, 34.962291) (138.656508, 35.141055)
89 66 (138.086119, 34.962291) (137.60373, 34.692986)
89 67 (138.086119, 34.962291) (137.970685, 34.743446)
89 68 (138.086119, 34.962291) (138.660796, 35.141761)
89 69 (138.086119, 34.962291) (138.920227, 35.011425)
89 70 (138.086119, 34.962291) (139.13835, 34.890952)
89 71 (1

91 27 (139.096934, 34.907863) (138.84221, 35.100529)
91 28 (139.096934, 34.907863) (137.602997, 34.688835)
91 29 (139.096934, 34.907863) (137.602661, 34.688835)
91 30 (139.096934, 34.907863) (138.965164, 35.311001)
91 31 (139.096934, 34.907863) (137.656326, 34.698334)
91 32 (139.096934, 34.907863) (137.645828, 34.698002)
91 33 (139.096934, 34.907863) (138.965164, 35.311001)
91 34 (139.096934, 34.907863) (138.390325, 34.973055)
91 35 (139.096934, 34.907863) (138.962173, 35.313835)
91 36 (139.096934, 34.907863) (138.963501, 35.310001)
91 37 (139.096934, 34.907863) (138.926496, 35.371694)
91 38 (139.096934, 34.907863) (138.926496, 35.371694)
91 39 (139.096934, 34.907863) (138.926668, 35.370959)
91 40 (139.096934, 34.907863) (138.52391205768, 34.995366104263)
91 41 (139.096934, 34.907863) (138.390569, 34.972062)
91 42 (139.096934, 34.907863) (138.967333, 35.307666)
91 43 (139.096934, 34.907863) (138.926496, 35.371694)
91 44 (139.096934, 34.907863) (138.52391205768, 34.995366104263)
91 45 (

92 86 (139.096934, 34.907863) (138.087661, 34.953336)
92 87 (139.096934, 34.907863) (138.442702, 35.001422)
92 88 (139.096934, 34.907863) (137.612427, 34.79008)
92 89 (139.096934, 34.907863) (138.086119, 34.962291)
92 90 (139.096934, 34.907863) (138.38813, 34.972567)
93 0 (138.086119, 34.962291) (138.922408, 35.014708)
93 1 (138.086119, 34.962291) (138.383231, 34.973188)
93 2 (138.086119, 34.962291) (138.45726, 34.996816)
93 3 (138.086119, 34.962291) (138.38908, 34.968594)
93 4 (138.086119, 34.962291) (138.417606, 34.982612)
93 5 (138.086119, 34.962291) (137.970685, 34.743446)
93 6 (138.086119, 34.962291) (138.462667, 34.980924)
93 7 (138.086119, 34.962291) (138.445501, 34.953985)
93 8 (138.086119, 34.962291) (138.606605, 35.120891)
93 9 (138.086119, 34.962291) (138.462667, 34.980924)
93 10 (138.086119, 34.962291) (138.462667, 34.980924)
93 11 (138.086119, 34.962291) (138.462667, 34.980924)
93 12 (138.086119, 34.962291) (138.462667, 34.980924)
93 13 (138.086119, 34.962291) (138.47949, 

In [11]:
import csv
with open('./shizuoka_distance.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(distance)

In [12]:
with open('./shizuoka_duration.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(duration)

In [7]:
startdistance = [0 for i in range(len(shizuoka_imgs))]
startduration = [0 for i in range(len(shizuoka_imgs))]
for i in range(len(shizuoka_imgs)):
    p1 = 34.9724, 138.3893
    p2 = shizuoka_imgs.iloc[i,1], shizuoka_imgs.iloc[i,2]
    p1r = tuple(reversed(p1))
    p2r = tuple(reversed(p2))
    if (p1r != p2r):
        # 経路計算 (Directions V2)
        print(i, p1r,p2r)
        routedict = client.directions((p1r, p2r))
        try:
            startdistance[i] = routedict["routes"][0]["summary"]["distance"]
            startduration[i] = routedict["routes"][0]["summary"]["duration"]
        except KeyError as instance:
            print(instance)
        time.sleep(1.5)
        index_i += 1
        if index_i // 1900 != index_key:
            index_key = index_i // 1900
            client = openrouteservice.Client(key=key[index_key])

0 (138.3893, 34.9724) (138.922408, 35.014708)
1 (138.3893, 34.9724) (138.383231, 34.973188)
2 (138.3893, 34.9724) (138.45726, 34.996816)
3 (138.3893, 34.9724) (138.38908, 34.968594)
4 (138.3893, 34.9724) (138.417606, 34.982612)
5 (138.3893, 34.9724) (137.970685, 34.743446)
6 (138.3893, 34.9724) (138.462667, 34.980924)
7 (138.3893, 34.9724) (138.445501, 34.953985)
8 (138.3893, 34.9724) (138.606605, 35.120891)
9 (138.3893, 34.9724) (138.462667, 34.980924)
10 (138.3893, 34.9724) (138.462667, 34.980924)
11 (138.3893, 34.9724) (138.462667, 34.980924)
12 (138.3893, 34.9724) (138.462667, 34.980924)
13 (138.3893, 34.9724) (138.47949, 35.022967)
14 (138.3893, 34.9724) (138.389633, 34.971105)
15 (138.3893, 34.9724) (138.389797, 34.972241)
16 (138.3893, 34.9724) (138.389811, 34.972277)
17 (138.3893, 34.9724) (138.462667, 34.980924)
18 (138.3893, 34.9724) (138.389224, 34.971519)
19 (138.3893, 34.9724) (138.388533, 34.972477)
20 (138.3893, 34.9724) (138.408508, 34.97586)
21 (138.3893, 34.9724) (138

In [12]:
with open('./shizuoka_startdistance.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(startdistance)

In [11]:
with open('./shizuoka_startduration.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(startduration)